<a href="https://colab.research.google.com/github/sakibreza/Emotion-Recognition-from-Multi-Channel-EEG-Signals/blob/main/sklearn_deap_ML_training_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
import pickle as pickle
import matplotlib.pyplot as plt
import os
import sys
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
import sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.io import loadmat 
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
torch.manual_seed(13)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_height = 9
input_width = 9
input_channel_num = 4
# conv_fuse = "plus"

# conv_1_shape = '4*4*1*16'
# pool_1_shape = 'None'

# # conv_2_shape = 'None'
# conv_2_shape = '4*4*1*32'
# pool_2_shape = 'None'

# # conv_3_shape = 'None'
# conv_3_shape = '4*4*1*64'
# pool_3_shape = 'None'

# conv_4_shape = '1*1*128*4'
# pool_4_shape = 'None'

time_step = 1
window_size = 1
# convolution full connected parameter
fc_size = 1024

dropout_prob = 0.5
np.random.seed(3)

calibration = 'N'
norm_type = '2D'
regularization_method = 'dropout'
enable_penalty = True

In [ ]:
arousal_key = "arousal_labels"
valence_key = "valence_labels"

# def minus(item):
#     return item-1

def construct_data(input_file, arousal_or_valence=None, bands=[0, 1, 2, 3]):
    input_channel_num = len(bands) * time_step

    dataset_dir = '/content/drive/MyDrive/DEAP_3D//DE3D_'
    ###load training set
    print("loading ",dataset_dir+input_file,".mat")
    data_file = sio.loadmat(dataset_dir+input_file+".mat")


    cnn_datasets = data_file["data"]

    #start
    arousal_labels = data_file[arousal_key]
    valence_labels = data_file[valence_key]

    arousal_labels = np.where(arousal_labels == 1, 2, arousal_labels)

    curr_labels = np.add(arousal_labels, valence_labels)

    # 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
    labels = curr_labels[0]

    # print('labels: ', np.unique(labels, return_counts= True))

    cnn_datasets = cnn_datasets.transpose(0,2,3,1)

    cnn_datasets = cnn_datasets[:,:,:,bands]

    cnn_datasets = cnn_datasets.reshape(len(cnn_datasets)//time_step, window_size,input_height,input_width,input_channel_num)

    index = np.array(range(0, len(labels)))
    np.random.shuffle(index)

    cnn_datasets   = cnn_datasets[index].reshape((-1, 4*9*9))
    labels  = labels[index]

    return cnn_datasets, labels  

In [ ]:
def multi_acc(y_test, y_pred):   
    acc = accuracy_score(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)
    return acc, conf_mat

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def report_result(best_model, data_X, data_y, label):
  print('\n',label,'\n----------------\n')
  nb_classes = 4
  data_y_hat = model.predict(data_X)
  data_acc, confusion_matrix = multi_acc(data_y, data_y_hat)
  print('****** mean classification accuracy: ', round(data_acc*100, 2), '% ******')
  # print(confusion_matrix)
  print('\n Accuracy per class\n ---------------')
  acc_per_class = np.diag(confusion_matrix)/np.sum(confusion_matrix, 1)

  for i in range(len(acc_per_class)):
    print(label_mapping[i], ': ', round(acc_per_class[i]*100, 2), '%')
  print('****** mean per class accuracy: ', round(np.mean(acc_per_class)*100, 2),'% ******\n')

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def get_result(best_model, data_X, data_y, label):
  ret = {}
  nb_classes = 4
  data_y_hat = model.predict(data_X)
  data_acc, confusion_matrix = multi_acc(data_y_hat, data_y)
  ret['acc'] = round(data_acc*100, 2)
  acc_per_class = np.diag(confusion_matrix)/np.sum(confusion_matrix, 1)
  for i in range(len(acc_per_class)):
    ret[label_mapping[i]] = round(acc_per_class[i]*100, 2)
  ret['per_class_acc'] = round(np.mean(acc_per_class)*100, 2)

  return ret

In [ ]:
{0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
acc_list = []
pc_acc_list = []
lalv_acc_list = []
lahv_acc_list = []
halv_acc_list = []
hahv_acc_list = []

for i in range(16):
  sub_id = i + 1
  print("**************** Process starting for subject no. ", sub_id, "**************** \n")
  if sub_id < 10:
    input_file = 's0' + str(sub_id)
  else:
    input_file = 's' + str(sub_id)
  data, labels = construct_data(input_file)
  X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
  model = LogisticRegression()
  model.fit(X_train, y_train)
  ret = get_result(model, X_test, y_test, 'Test Data')
  print('accuracy: ', ret['acc'], '%')
  acc_list.append(ret['acc'])
  pc_acc_list.append(ret['per_class_acc'])
  lalv_acc_list.append(ret['LALV'])
  lahv_acc_list.append(ret['LAHV'])
  halv_acc_list.append(ret['HALV'])
  hahv_acc_list.append(ret['HAHV'])

**************** Process starting for subject no.  1 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s01 .mat


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy:  82.71 %
**************** Process starting for subject no.  2 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s02 .mat


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy:  80.21 %
**************** Process starting for subject no.  3 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s03 .mat


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy:  83.12 %
**************** Process starting for subject no.  15 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s15 .mat


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy:  85.21 %
**************** Process starting for subject no.  16 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s16 .mat
accuracy:  79.79 %


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
result_dict =   {'acc': acc_list,
  'pc_acc': pc_acc_list,
  'lalv': lalv_acc_list,
  'lahv': lahv_acc_list,
  'halv': halv_acc_list,
  'hahv': hahv_acc_list}

In [ ]:
np.mean(np.array(result_dict['acc']))

82.539375

In [ ]:
result_dict

{'acc': [82.71,
  80.21,
  79.79,
  84.38,
  78.96,
  86.25,
  84.79,
  78.54,
  81.25,
  84.17,
  78.54,
  81.67,
  91.25,
  83.12,
  85.21,
  79.79],
 'hahv': [85.9,
  81.55,
  86.67,
  81.69,
  81.76,
  85.62,
  89.71,
  80.23,
  84.46,
  85.71,
  84.09,
  85.09,
  92.71,
  79.64,
  84.76,
  80.17],
 'halv': [83.08,
  74.74,
  81.82,
  80.0,
  76.92,
  87.72,
  78.02,
  76.8,
  76.14,
  86.43,
  82.61,
  78.21,
  90.39,
  88.0,
  82.44,
  80.65],
 'lahv': [77.92,
  78.95,
  80.58,
  84.44,
  75.36,
  86.27,
  78.63,
  81.48,
  76.47,
  84.76,
  74.74,
  77.78,
  85.71,
  81.91,
  86.76,
  86.15],
 'lalv': [81.2,
  82.79,
  76.52,
  86.83,
  80.0,
  86.36,
  89.71,
  75.49,
  82.44,
  78.9,
  78.2,
  80.0,
  91.11,
  79.55,
  87.04,
  76.47],
 'pc_acc': [82.02,
  79.51,
  81.39,
  83.24,
  78.51,
  86.49,
  84.02,
  78.5,
  79.88,
  83.95,
  79.91,
  80.27,
  89.98,
  82.28,
  85.25,
  80.86]}

In [ ]:
np.save('result_logistic.npy', result_dict)